In [5]:
import json
import pandas as pd
import numpy as np

In [6]:
status_file_name = r'D:\Learning\ML\BTL\preprocessing\clean_data\status_per_min.json'

In [7]:
df = pd.read_json(status_file_name)

In [8]:
# Calculate gold transition matrix
gold_columns_to_select = [col for col in df.columns if col.startswith('gold_difference_at_') and int(col.split('_')[-1]) >= 20]
df_selected = df[gold_columns_to_select]
n_states = 51
n_columns = df_selected.shape[1]
gold_transition_matrix = np.zeros((n_states, n_states))
for i in range(n_columns - 1):
        for j in range(df_selected.shape[0]):
            # Kiểm tra nếu giá trị tại (j, i) không phải là NaN
            if not pd.isna(df_selected.iloc[j, i]) and not pd.isna(df_selected.iloc[j, i + 1]):
                current_state = df_selected.iloc[j, i]

                next_state = df_selected.iloc[j, i + 1]

                # Chuyển đổi giá trị thành chỉ mục hợp lệ (bù 25 để chuyển về giá trị dương)
                current_state_bin = int(current_state + 25)  # Bù 25 để giá trị từ -25 đến 25 thành 0 đến 50
                next_state_bin = int(next_state + 25)

                # Cập nhật ma trận chuyển tiếp
                gold_transition_matrix[current_state_bin, next_state_bin] += 1
gold_transition_matrix += 1
row_sums = gold_transition_matrix.sum(axis=1, keepdims=True)
gold_transition_matrix = gold_transition_matrix / row_sums

In [9]:
# Fill gold NaN
for index, row in df[gold_columns_to_select].iterrows():
    for minute in range(20, 41):
        col = f"gold_difference_at_{minute}"
        if pd.isna(row[col]):
            # Lấy trạng thái gần nhất
            prev_minute = minute - 1
            while prev_minute >= 20 and pd.isna(row[f"gold_difference_at_{prev_minute}"]):
                prev_minute -= 1
            
            if prev_minute >= 20:
                # Trạng thái trước đó
                prev_value = row[f"gold_difference_at_{prev_minute}"]
                
                # Chọn trạng thái tiếp theo dựa trên ma trận chuyển tiếp
                probabilities = gold_transition_matrix[int(prev_value)]
                df.at[index, col] = np.random.choice(len(probabilities), p=probabilities)
            else:
                break

In [10]:
# Fild level NaN
roles = ["TOP", "JUNGLE", "MID", "BOT", "SUPPORT"]
level_columns_to_select = [col for col in df.columns if col.startswith('level_difference_') and int(col.split('_')[-1]) >= 20]
for role in roles:
        for minute in range(21, 41):
            col = f"level_difference_{role}_at_{minute}"
            prev_col = f"level_difference_{role}_at_{minute - 1}"
            # Nếu cột hiện tại có NaN, điền giá trị từ phút trước
            df[col] = df[col].fillna(df[prev_col])

In [11]:
# Fill turret diff
teams = ["blue", "red"]
lanes = ["MID_LANE", "TOP_LANE", "BOT_LANE"]
for minute in range(20, 41):
        for team in teams:
            for lane in lanes:
                col = f"{team}_{lane}_TURRETS_at_{minute}"
                # Xác định tên cột phút trước (minute - 1)
                prev_col = col.replace(f"at_{minute}", f"at_{minute - 1}")
                
                # Nếu giá trị ở phút x là NaN, điền giá trị từ phút trước (minute - 1)
                df[col] = df[col].fillna(df[prev_col])

In [12]:
# Fill monster diff
elite_monsters = ["HORDE", "RIFTHERALD", "FIRE_DRAGON", "WATER_DRAGON", "EARTH_DRAGON", 
                 "AIR_DRAGON", "CHEMTECH_DRAGON", "HEXTECH_DRAGON", "ELDER_DRAGON", "BARON_NASHOR"]
teams = ["blue", "red"]
for minute in range(20, 41):
    for elite_monster in elite_monsters:
        col = f"{elite_monster}_diff_at_{minute}"
        prev_col = col.replace(f"at_{minute}", f"at_{minute - 1}")
        df[col] = df[col].fillna(df[prev_col].astype(int))

In [13]:
df.to_csv(r'D:\Learning\ML\BTL\preprocessing\clean_data\status_per_min.csv', index=False, encoding='utf-8')